In [ ]:
import pickle
import numpy as np
from src.groupedpaneldatamodels import GroupedFixedEffects, GroupedInteractiveFixedEffects
from joblib import Parallel, delayed
import cProfile

# Bonhomme Manresa

In [4]:
def bm_run(run, G, i):
    x, y, f, g = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    f = np.float32(f)
    gfe_iterations = 10 if G < 4 else 100
    n_boot = 200 if G < 4 else 100
    m = GroupedFixedEffects(y, x, G, True, hide_progressbar=True).fit(
        max_iter=100, gfe_iterations=gfe_iterations, n_boot=n_boot
    )
    print(f"Finished run {i}")

    return m.to_dict()

In [5]:
for n in (100, 200,):
    for t in (20, 50,):
        for G in (3, 6,):
            for k in (3,):
                print(f"\n\nRunning DGP1 with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp1_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))
                estimates = []
                estimates = Parallel(n_jobs=-1, prefer="processes")(
                    delayed(bm_run)(run, G, i) for i, run in enumerate(runs[:200])
                )

                with open(f"estimates/dgp1_n{n}_t{t}_G{G}_k{k}.pkl", "wb") as f:
                    pickle.dump(estimates, f)



Running DGP1 with n=100, t=20, G=3, k=3
Finished run 2
Finished run 1
Finished run 3
Finished run 4
Finished run 0
Finished run 6
Finished run 8
Finished run 7
Finished run 5
Finished run 9
Finished run 12
Finished run 13
Finished run 19
Finished run 10
Finished run 17
Finished run 16
Finished run 11
Finished run 15
Finished run 18
Finished run 14
Finished run 20
Finished run 24
Finished run 27
Finished run 21
Finished run 22
Finished run 23
Finished run 28
Finished run 26
Finished run 25
Finished run 29
Finished run 30
Finished run 31
Finished run 35
Finished run 33
Finished run 32
Finished run 34
Finished run 38
Finished run 36
Finished run 37
Finished run 39
Finished run 40
Finished run 41
Finished run 43
Finished run 45
Finished run 42
Finished run 47
Finished run 44
Finished run 48
Finished run 49
Finished run 46
Finished run 50
Finished run 52
Finished run 54
Finished run 51
Finished run 55
Finished run 53
Finished run 56
Finished run 57
Finished run 59
Finished run 58
Finished

KeyboardInterrupt: 

In [ ]:
def ab_run(run, G, i):
    x, y, f, g, *_ = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    f = np.float32(f)
    gife_iterations = 10 if G < 4 else 100
    n_boot = 200 if G < 4 else 100
    m = GroupedInteractiveFixedEffects(y, x, G, True, hide_progressbar=True).fit(max_iter=100, gife_iterations=gife_iterations, n_boot=n_boot)
    print(f"Finished run {i}")

    return m.to_dict()


for n in (100, 200):
    for t in (20, 50):
        for G in (3, 6):
            for k in (3,):
                print(f"\n\nRunning DGP2 (ab) with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp2_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))
                estimates = []
                estimates = Parallel(n_jobs=-1, prefer="processes")(
                    delayed(ab_run)(run, G, i) for i, run in enumerate(runs[:200])
                )

                with open(f"estimates/dgp2_n{n}_t{t}_G{G}_k{k}.pkl", "wb") as f:
                    pickle.dump(estimates, f)

In [ ]:
def sj_run(run, G, i):
    x, y, f, g, *_ = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    f = np.float32(f)
    n_boot = 100 if G < 4 else 50
    kappa = 0.005 if G < 4 else 0.00005
    m = GroupedInteractiveFixedEffects(y, x, G, True, "su_ju", R=G, hide_progressbar=True).fit(
        n_boot=n_boot, only_bfgs=True, max_iter=100, kappa=kappa, tol=1e-2
    )
    print(f"Finished run {i}")

    return m.to_dict()

for n in (100, 200):
    for t in (20, 50):
        for G in (3, 6):
            for k in (3,):
                print(f"\n\nRunning DGP2 (su_ju) with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp2_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))
                estimates = []
                estimates = Parallel(n_jobs=-1, prefer="processes")(
                    delayed(sj_run)(run, G, i) for i, run in enumerate(runs[:200])
                )

                with open(f"estimates/dgp2_n{n}_t{t}_G{G}_k{k}_su_ju.pkl", "wb") as f:
                    pickle.dump(estimates, f)

In [ ]:
def ssp_run(run, G, i):
    x, y, g = run
    x = np.float32(x)
    y = np.atleast_3d(np.float32(y))
    n_boot = 100 if G < 4 else 50
    kappa = 0.005 if G < 4 else 0.00005
    m = GroupedFixedEffects(y, x, G, True, "su_shi_phillips", hide_progressbar=True).fit(
        n_boot=n_boot, only_bfgs=True, max_iter=100, kappa=kappa, tol=1e-2
    )

    print(f"Finished run {i}")
    return m.to_dict()

for n in (100, 200):
    for t in (20, 50):
        for G in (3, 6):
            for k in (3,):
                print(f"\n\nRunning DGP3 (su_shi_phillips) with n={n}, t={t}, G={G}, k={k}")
                runs = pickle.load(open(f"development/generated_data/dgp3_n{n}_t{t}_G{G}_k{k}.pkl", "rb"))
                estimates = []
                estimates = Parallel(n_jobs=-1, prefer="processes")(
                    delayed(ssp_run)(run, G, i) for i, run in enumerate(runs[:200])
                )
                with open(f"estimates/dgp3_n{n}_t{t}_G{G}_k{k}.pkl", "wb") as f:
                    pickle.dump(estimates, f)